##### Copyright 2020 The OpenFermion Developers

# Hamiltonian Time Evolution and Expectation Value Computation

This tutorial describes the FQE's capabilities for Hamiltonian time-evolution and expectation value estimation

Where possible, LiH will be used as an example molecule for the API.

In [1]:
Print = True
from openfermion import FermionOperator, MolecularData
from openfermion.utils import hermitian_conjugated
import numpy
import fqe
from fqe.unittest_data import build_lih_data, build_hamiltonian
numpy.set_printoptions(floatmode='fixed', precision=6, linewidth=80, suppress=True)
numpy.random.seed(seed=409)

h1e, h2e, wfn = build_lih_data.build_lih_data('energy')
lih_hamiltonian = fqe.get_restricted_hamiltonian(([h1e, h2e]))
lihwfn = fqe.Wavefunction([[4, 0, 6]])
lihwfn.set_wfn(strategy='from_data', raw_data={(4, 0): wfn})
if Print:
    lihwfn.print_wfn()


Sector N = 4 : S_z = 0
a'000011'b'000011' (-0.9870890035778126+0j)
a'000011'b'000101' (-0.0384854879340124+0j)
a'000011'b'100001' (-0.0036477466809967+0j)
a'000101'b'000011' (-0.0384854879340073+0j)
a'000101'b'000101' (0.0344508540105218+0j)
a'000101'b'100001' (-0.0592953695921639+0j)
a'000101'b'100010' (-0.0010777697106179+0j)
a'001001'b'001001' (0.0271656962509733+0j)
a'001001'b'001010' (-0.0031209939957822+0j)
a'010001'b'010001' (0.0271656962509733+0j)
a'010001'b'010010' (-0.0031209939957822+0j)
a'100001'b'000011' (-0.003647746680997+0j)
a'100001'b'000101' (-0.0592953695921649+0j)
a'100001'b'100001' (0.1135892009408387+0j)
a'000110'b'000110' (0.0037623492033852+0j)
a'001010'b'001001' (-0.0031209939957822+0j)
a'001010'b'001010' (0.001794202149606+0j)
a'010010'b'010001' (-0.0031209939957822+0j)
a'010010'b'010010' (0.001794202149606+0j)
a'100010'b'000101' (-0.001077769710618+0j)
a'100010'b'100010' (0.0010515758855392+0j)


## Application of one- and two-body fermionic gates

The API for time propogation can be invoked through the fqe namespace or the wavefunction object

In [2]:
# dummy geometry
from openfermion.chem.molecular_data import spinorb_from_spatial
from openfermion import jordan_wigner, get_sparse_operator, InteractionOperator, get_fermion_operator

h1s, h2s = spinorb_from_spatial(h1e, numpy.einsum("ijlk", -2 * h2e) * 0.5)
mol = InteractionOperator(0, h1s, h2s)
ham_fop = get_fermion_operator(mol)
ham_mat = get_sparse_operator(jordan_wigner(ham_fop)).toarray()


In [3]:
from scipy.linalg import expm
time = 0.01
evolved1 = lihwfn.time_evolve(time, lih_hamiltonian)
if Print:
    evolved1.print_wfn()
evolved2 = fqe.time_evolve(lihwfn, time, lih_hamiltonian)
if Print:
    evolved2.print_wfn()
assert numpy.isclose(fqe.vdot(evolved1, evolved2), 1)
cirq_wf = fqe.to_cirq_ncr(lihwfn)
evolve_cirq = expm(-1j * time * ham_mat) @ cirq_wf
test_evolve = fqe.from_cirq(evolve_cirq, thresh=1.0E-12)
assert numpy.isclose(fqe.vdot(test_evolve, evolved1), 1)

Sector N = 4 : S_z = 0
a'000011'b'000011' (-0.9832017407384288-0.08751592993809962j)
a'000011'b'000101' (-0.038333927909412015-0.0034121477949351545j)
a'000011'b'100001' (-0.003633381455202132-0.00032341158146809j)
a'000101'b'000011' (-0.038333927909406935-0.0034121477949347087j)
a'000101'b'000101' (0.03431518282571416+0.003054434433325243j)
a'000101'b'100001' (-0.05906185803441181-0.005257164690403509j)
a'000101'b'100010' (-0.001073525344578889-9.555564554363788e-05j)
a'001001'b'001001' (0.02705871481472957+0.0024085276354275376j)
a'001001'b'001010' (-0.003108703183821363-0.00027670930255657017j)
a'010001'b'010001' (0.02705871481472957+0.0024085276354275376j)
a'010001'b'010010' (-0.003108703183821363-0.00027670930255657017j)
a'100001'b'000011' (-0.0036333814552024306-0.0003234115814681175j)
a'100001'b'000101' (-0.0590618580344128-0.005257164690403599j)
a'100001'b'100001' (0.11314187441703756+0.010070889930469399j)
a'000110'b'000110' (0.003747532670094563+0.00033357218982965186j)
a'001

## Exact evolution implementation of quadratic Hamiltonians

Listed here are examples of evolving the special Hamiltonians.

Diagonal Hamiltonian evolution is supported.

In [4]:
wfn = fqe.Wavefunction([[4, 2, 4]])
wfn.set_wfn(strategy='random')
if Print:
    wfn.print_wfn()

diagonal = FermionOperator('0^ 0', -2.0) + \
           FermionOperator('1^ 1', -1.7) + \
           FermionOperator('2^ 2', -0.7) + \
           FermionOperator('3^ 3', -0.55) + \
           FermionOperator('4^ 4', -0.1) + \
           FermionOperator('5^ 5', -0.06) + \
           FermionOperator('6^ 6', 0.5) + \
           FermionOperator('7^ 7', 0.3)
if Print:
    print(diagonal)
    
evolved = wfn.time_evolve(time, diagonal)
if Print:
    evolved.print_wfn()

Sector N = 4 : S_z = 2
a'0111'b'0001' (-0.2573172917766764-0.022065525915195058j)
a'0111'b'0010' (-0.19069981498041647-0.1574408028420647j)
a'0111'b'0100' (-0.13843008024130438+0.6019949651839999j)
a'0111'b'1000' (-0.1006924842310149-0.2625890797363573j)
a'1011'b'0001' (0.017019416043439945+0.11531103507659801j)
a'1011'b'0010' (-0.24705424991147376-0.16189142882301666j)
a'1011'b'0100' (-0.22886695583773378+0.1634347611565236j)
a'1011'b'1000' (-0.1711253096436585+0.1488721298012593j)
a'1101'b'0001' (0.1473565643305041-0.12158475184832697j)
a'1101'b'0010' (0.16112138688490701+0.001203459546465527j)
a'1101'b'0100' (0.05261727070820834-0.08021022138631807j)
a'1101'b'1000' (-0.05366446767497438-0.03981879197579796j)
a'1110'b'0001' (0.007163733471580563-0.11642659346027902j)
a'1110'b'0010' (0.12513118505662785+0.0005967634674837373j)
a'1110'b'0100' (-0.021197557000183945+0.10844705229527415j)
a'1110'b'1000' (-0.043769805448620906-0.24618852439514735j)
-2.0 [0^ 0] +
-1.7 [1^ 1] +
-0.7 [2^ 2] 

Exact evolution of dense quadratic hamiltonians is supported.  Here is an evolution example using a spin restricted Hamiltonian on a number and spin conserving wavefunction

In [5]:
norb = 4 
h1e = numpy.zeros((norb, norb), dtype=numpy.complex128) 
for i in range(norb): 
    for j in range(norb): 
        h1e[i, j] += (i+j) * 0.02 
    h1e[i, i] += i * 2.0 

hamil = fqe.get_restricted_hamiltonian((h1e,)) 
wfn = fqe.Wavefunction([[4, 0, norb]]) 
wfn.set_wfn(strategy='random') 
initial_energy = wfn.expectationValue(hamil) 
print('Initial Energy: {}'.format(initial_energy))
evolved = wfn.time_evolve(time, hamil) 
final_energy = evolved.expectationValue(hamil)
print('Final Energy:   {}'.format(final_energy))

Initial Energy: (12.630481923700247-1.1102230246251565e-16j)
Final Energy:   (12.630481923700177-1.1102230246251565e-16j)


The GSO Hamiltonian is for evolution of quadratic hamiltonians that are spin broken and number conserving.

In [6]:
norb = 4 
h1e = numpy.zeros((2*norb, 2*norb), dtype=numpy.complex128) 
for i in range(2*norb): 
    for j in range(2*norb): 
        h1e[i, j] += (i+j) * 0.02 
    h1e[i, i] += i * 2.0 

hamil = fqe.get_gso_hamiltonian((h1e,)) 
wfn = fqe.get_number_conserving_wavefunction(4, norb) 
wfn.set_wfn(strategy='random') 
initial_energy = wfn.expectationValue(hamil) 
print('Initial Energy: {}'.format(initial_energy))
evolved = wfn.time_evolve(time, hamil) 
final_energy = evolved.expectationValue(hamil)
print('Final Energy:   {}'.format(final_energy))

Initial Energy: (141.2704138222322+1.5543122344752192e-15j)
Final Energy:   (141.2704138222318+2.4424906541753444e-15j)


The BCS hamiltonian evovles spin conserved and number broken wavefunctions.

In [7]:
norb = 4
time = 0.001
wfn_spin = fqe.get_spin_conserving_wavefunction(2, norb)
hamil = FermionOperator('', 6.0)
for i in range(0, 2*norb, 2):
    for j in range(0, 2*norb, 2):
        opstring = str(i) + ' ' + str(j + 1)
        hamil += FermionOperator(opstring, (i+1 + j*2)*0.1 - (i+1 + 2*(j + 1))*0.1j)
        opstring = str(i) + '^ ' + str(j + 1) + '^ '
        hamil += FermionOperator(opstring, (i+1 + j)*0.1 + (i+1 + j)*0.1j)
h_noncon = (hamil + hermitian_conjugated(hamil))/2.0
if Print:
    print(h_noncon)

wfn_spin.set_wfn(strategy='random')
if Print:
    wfn_spin.print_wfn()

spin_evolved = wfn_spin.time_evolve(time, h_noncon)
if Print:
    spin_evolved.print_wfn()

6.0 [] +
(0.05-0.15000000000000002j) [0 1] +
(0.25-0.35000000000000003j) [0 3] +
(0.45-0.55j) [0 5] +
(0.65-0.75j) [0 7] +
(0.05+0.05j) [0^ 1^] +
(0.15000000000000002+0.15000000000000002j) [0^ 3^] +
(0.25+0.25j) [0^ 5^] +
(0.35000000000000003+0.35000000000000003j) [0^ 7^] +
(0.05-0.05j) [1 0] +
(0.15000000000000002-0.15000000000000002j) [1 2] +
(0.25-0.25j) [1 4] +
(0.35000000000000003-0.35000000000000003j) [1 6] +
(0.05+0.15000000000000002j) [1^ 0^] +
(0.15000000000000002+0.25j) [1^ 2^] +
(0.25+0.35000000000000003j) [1^ 4^] +
(0.35000000000000003+0.45j) [1^ 6^] +
(0.15000000000000002-0.25j) [2 1] +
(0.35000000000000003-0.45j) [2 3] +
(0.55-0.65j) [2 5] +
(0.75-0.8500000000000001j) [2 7] +
(0.15000000000000002+0.15000000000000002j) [2^ 1^] +
(0.25+0.25j) [2^ 3^] +
(0.35000000000000003+0.35000000000000003j) [2^ 5^] +
(0.45+0.45j) [2^ 7^] +
(0.15000000000000002-0.15000000000000002j) [3 0] +
(0.25-0.25j) [3 2] +
(0.35000000000000003-0.35000000000000003j) [3 4] +
(0.45-0.45j) [3 6] +
(0.25

Exact Evolution Implementation of Diagonal Coulomb terms

In [8]:
norb = 4
wfn = fqe.Wavefunction([[5, 1, norb]])
vij = numpy.zeros((norb, norb, norb, norb), dtype=numpy.complex128)
for i in range(norb):
            for j in range(norb):
                vij[i, j] += 4*(i % norb + 1)*(j % norb + 1)*0.21
                
wfn.set_wfn(strategy='random')

if Print:
    wfn.print_wfn()
    
hamil = fqe.get_diagonalcoulomb_hamiltonian(vij)
    
evolved = wfn.time_evolve(time, hamil)
if Print:
    evolved.print_wfn()

Sector N = 5 : S_z = 1
a'0111'b'0011' (-0.17044188783239003+0.015549713711555089j)
a'0111'b'0101' (-0.016353688142264605-0.023995465888992637j)
a'0111'b'1001' (0.026411679481048472+0.2550148809272792j)
a'0111'b'0110' (-0.01772081263333609+0.2742953249494439j)
a'0111'b'1010' (-0.04089100193038861-0.01830297061912792j)
a'0111'b'1100' (-0.12205984401721448-0.16716321188705718j)
a'1011'b'0011' (0.033665463396725874-0.1003209112144722j)
a'1011'b'0101' (-0.1501205613922792+0.22859967143937296j)
a'1011'b'1001' (0.003508701856054892-0.23585969152969283j)
a'1011'b'0110' (0.13625386157816874+0.01089320887489491j)
a'1011'b'1010' (-0.11993673806176988-0.12168361132300079j)
a'1011'b'1100' (-0.00968137721753541-0.0800905544423775j)
a'1101'b'0011' (0.06694907741954417+0.17089304859301946j)
a'1101'b'0101' (0.044962462688345285-0.03544318445969587j)
a'1101'b'1001' (0.39212411149643445-0.1046844731489852j)
a'1101'b'0110' (-0.00620034572910763+0.15757970448407252j)
a'1101'b'1010' (-0.1259386504469936+0.0

Exact evolution of individual n-body anti-Hermitian gnerators

In [9]:
norb = 3
nele = 4
ops = FermionOperator('5^ 1^ 2 0', 3.0 - 1.j)
ops += FermionOperator('0^ 2^ 1 5', 3.0 + 1.j)
wfn = fqe.get_number_conserving_wavefunction(nele, norb)
wfn.set_wfn(strategy='random')
wfn.normalize()
if Print:
    wfn.print_wfn()
evolved = wfn.time_evolve(time, ops)
if Print:
    evolved.print_wfn()


Sector N = 4 : S_z = -2
a'001'b'111' (0.30390798276421177+0.03849022160763121j)
a'010'b'111' (0.022704974112154053+0.15213535840863748j)
a'100'b'111' (-0.4635735659147738+0.030506393487579976j)
Sector N = 4 : S_z = 0
a'011'b'011' (-0.06440203545045875+0.24290231417536573j)
a'011'b'101' (-0.013860104216657742+0.178496005054772j)
a'011'b'110' (-0.14901953372764978+0.10984353579388344j)
a'101'b'011' (0.14378938398839428+0.1424595738811093j)
a'101'b'101' (-0.06774238214202746+0.2264174569173502j)
a'101'b'110' (-0.21178860815906608+0.04132932830693203j)
a'110'b'011' (-0.08297937926453706-0.1446977831246778j)
a'110'b'101' (0.05334564621933154-0.023307424061958064j)
a'110'b'110' (-0.05535596548738161-0.16185173434343056j)
Sector N = 4 : S_z = 2
a'111'b'001' (-0.12666672895403738+0.05116497678766766j)
a'111'b'010' (-0.06420819321126867-0.3771059438035301j)
a'111'b'100' (-0.38050213630121726+0.15348471455665633j)
Sector N = 4 : S_z = -2
a'001'b'111' (0.30390798276421177+0.03849022160763121j)
a'

 Approximate evolution of sums of n-body generators

Approximate evolution can be done for dense operators.

In [10]:
lih_evolved = lihwfn.apply_generated_unitary(time, 'taylor', lih_hamiltonian, accuracy=1.e-8)
if Print:
    lih_evolved.print_wfn()

Sector N = 4 : S_z = 0
a'000011'b'000011' (-0.9870501056639676-0.00876298425084819j)
a'000011'b'000101' (-0.0384839713479148-0.0003416589118862754j)
a'000011'b'100001' (-0.003647602935297334-3.238325409440235e-05j)
a'000101'b'000011' (-0.0384839713479097-0.0003416589118862308j)
a'000101'b'000101' (0.03444949641617426+0.0003058410149547989j)
a'000101'b'100001' (-0.059293032957665366-0.0005264007526380628j)
a'000101'b'100010' (-0.0010777272393492372-9.568002262034465e-06j)
a'001001'b'001001' (0.02716462574072948+0.00024116626254639198j)
a'001001'b'001010' (-0.003120871007714629-2.770694732178501e-05j)
a'010001'b'010001' (0.02716462574072948+0.000241166262546392j)
a'010001'b'010010' (-0.003120871007714629-2.770694732178501e-05j)
a'100001'b'000011' (-0.0036476029352976343-3.2383254094405115e-05j)
a'100001'b'000101' (-0.059293032957666365-0.0005264007526380719j)
a'100001'b'100001' (0.11358472476587893+0.0010083998412408193j)
a'000110'b'000110' (0.0037622009416752988+3.340063743457013e-05j)


In [11]:
norb = 2
nalpha = 1
nbeta = 1
nele = nalpha + nbeta
time = 0.05
h1e = numpy.zeros((norb*2, norb*2), dtype=numpy.complex128)
for i in range(2*norb):
    for j in range(2*norb):
        h1e[i, j] += (i+j) * 0.02
    h1e[i, i] += i * 2.0
hamil = fqe.get_general_hamiltonian((h1e,))
spec_lim = [-1.13199078e-03, 6.12720338e+00]
wfn = fqe.Wavefunction([[nele, nalpha - nbeta, norb]])
wfn.set_wfn(strategy='random')
if Print:
    wfn.print_wfn()
evol_wfn = wfn.apply_generated_unitary(time, 'chebyshev', hamil, spec_lim=spec_lim)
if Print:
    evol_wfn.print_wfn()

Sector N = 2 : S_z = 0
a'01'b'01' (0.5908062711142874-0.14725857523270403j)
a'01'b'10' (-0.003471410086752633+0.02161237923081287j)
a'10'b'01' (0.6457522689404723+0.21530201001447957j)
a'10'b'10' (-0.013918280398179678-0.40649606335261473j)
Sector N = 2 : S_z = 0
a'01'b'01' (0.5488705264238126-0.2645790387917878j)
a'01'b'10' (0.0013642586550375838+0.01913723432686206j)
a'10'b'01' (0.6784342247800093+0.011010414872908991j)
a'10'b'10' (-0.17416169432186263-0.3710046632568297j)


API for determining desired expectation values

In [12]:
rdm1 = lihwfn.expectationValue('i^ j')
if Print:
    print(rdm1)
val = lihwfn.expectationValue('5^ 3')
if Print:
    print(2.*val)
trdm1 = fqe.expectationValue(lih_evolved, 'i j^', lihwfn)
if Print:
    print(trdm1)
val = fqe.expectationValue(lih_evolved, '5 3^', lihwfn)
if Print:
    print(2*val)

[[ 1.999908+0.000000j -0.000284+0.000000j  0.000441+0.000000j
  -0.000000+0.000000j -0.000000+0.000000j -0.001285+0.000000j]
 [-0.000284+0.000000j  1.951766+0.000000j  0.073757+0.000000j
   0.000000+0.000000j  0.000000+0.000000j  0.010948+0.000000j]
 [ 0.000441+0.000000j  0.073757+0.000000j  0.012402+0.000000j
   0.000000+0.000000j  0.000000+0.000000j -0.017277+0.000000j]
 [-0.000000+0.000000j  0.000000+0.000000j  0.000000+0.000000j
   0.001525+0.000000j  0.000000+0.000000j  0.000000+0.000000j]
 [-0.000000+0.000000j  0.000000+0.000000j  0.000000+0.000000j
   0.000000+0.000000j  0.001525+0.000000j -0.000000+0.000000j]
 [-0.001285+0.000000j  0.010948+0.000000j -0.017277+0.000000j
   0.000000+0.000000j -0.000000+0.000000j  0.032874+0.000000j]]
(0.0737569627134323+0j)
[[ 0.000170-0.017754j  0.000284+0.000003j -0.000441-0.000004j
   0.000000+0.000000j  0.000000+0.000000j  0.001285+0.000011j]
 [ 0.000284+0.000003j  0.048311-0.017327j -0.073754-0.000655j
  -0.000000-0.000000j -0.000000-0.0000

2.B.1 RDMs 
In addition to the above API higher order density matrices in addition to hole densities can be calculated.

In [13]:
rdm2 = lihwfn.expectationValue('i^ j k l^')
if Print:
    print(rdm2)
rdm2 = fqe.expectationValue(lihwfn, 'i^ j^ k l', lihwfn)
if Print:
    print(rdm2)

[[[[-0.000047+0.000000j -0.000260+0.000000j  0.000445+0.000000j
    -0.000000+0.000000j -0.000000+0.000000j -0.001287+0.000000j]
   [ 0.000308+0.000000j  0.048269+0.000000j -0.073761+0.000000j
    -0.000000+0.000000j -0.000000+0.000000j -0.010927+0.000000j]
   [-0.000437+0.000000j -0.073761+0.000000j  1.987537+0.000000j
    -0.000000+0.000000j -0.000000+0.000000j  0.017274+0.000000j]
   [ 0.000000+0.000000j -0.000000+0.000000j -0.000000+0.000000j
     1.998412+0.000000j -0.000000+0.000000j -0.000000+0.000000j]
   [ 0.000000+0.000000j -0.000000+0.000000j -0.000000+0.000000j
    -0.000000+0.000000j  1.998412+0.000000j  0.000000+0.000000j]
   [ 0.001283+0.000000j -0.010927+0.000000j  0.017274+0.000000j
    -0.000000+0.000000j  0.000000+0.000000j  1.967051+0.000000j]]

  [[-0.000544+0.000000j -0.096416+0.000000j  0.147514+0.000000j
     0.000000+0.000000j  0.000000+0.000000j  0.021885+0.000000j]
   [ 0.000733+0.000000j  0.000242+0.000000j  0.000079+0.000000j
     0.000000+0.000000j  0.0000

     0.000000+0.000000j -0.000000+0.000000j -0.007066+0.000000j]]]]
[[[[-1.999862+0.000000j  0.000260+0.000000j -0.000445+0.000000j
     0.000000+0.000000j  0.000000+0.000000j  0.001287+0.000000j]
   [ 0.000260+0.000000j -0.000733+0.000000j -0.000282+0.000000j
    -0.000000+0.000000j -0.000000+0.000000j  0.000012+0.000000j]
   [-0.000445+0.000000j -0.000282+0.000000j  0.007548+0.000000j
    -0.000000+0.000000j -0.000000+0.000000j -0.000293+0.000000j]
   [ 0.000000+0.000000j -0.000000+0.000000j -0.000000+0.000000j
     0.003666+0.000000j  0.000000+0.000000j -0.000000+0.000000j]
   [ 0.000000+0.000000j -0.000000+0.000000j -0.000000+0.000000j
     0.000000+0.000000j  0.003666+0.000000j  0.000000+0.000000j]
   [ 0.001287+0.000000j  0.000012+0.000000j -0.000293+0.000000j
    -0.000000+0.000000j  0.000000+0.000000j  0.002124+0.000000j]]

  [[ 0.000260+0.000000j -3.903401+0.000000j -0.147514+0.000000j
    -0.000000+0.000000j -0.000000+0.000000j -0.021885+0.000000j]
   [ 1.951639+0.000000j  0.

2.B.2 Hamiltonian expectations (or any expectation values)

In [14]:
li_h_energy = lihwfn.expectationValue(lih_hamiltonian)
if Print:
    print(li_h_energy)
li_h_energy = fqe.expectationValue(lihwfn, lih_hamiltonian, lihwfn)
if Print:
    print(li_h_energy)

(-8.87771957038547+0j)
(-8.87771957038547+0j)


2.B.3 Symmetry operations

In [15]:
op = fqe.get_s2_operator()
print(lihwfn.expectationValue(op))
op = fqe.get_sz_operator()
print(lihwfn.expectationValue(op))
op = fqe.get_time_reversal_operator()
print(lihwfn.expectationValue(op))
op = fqe.get_number_operator()
print(lihwfn.expectationValue(op))


(-1.896911147578357e-23+0j)
0j
(1.0000000000000018+0j)
(4.000000000000007+0j)
